In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
import re
import sys
from collections import Counter
from functools import reduce
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rich import print
from tqdm import tqdm

In [2]:
def read_file_to_str_li(fp, print_exp=True):
    with open(fp, "r") as f:
        lines = f.read().split("\n")
    if print_exp:
        print(f"Read from {fp}:")
        print(f"First line: {lines[0]} | Last line: {lines[-1]}")
        print("-" * 6)

    return lines


# define the function blocks
def convert_to_int(input_str):
    if input_str == "" or input_str == " ":
        return None
    return int(input_str)


def convert_to_str(input_str):
    return str(input_str)


# map the inputs to the function blocks
converts = {
    "i": convert_to_int,
    "s": convert_to_str,
}


def convert_str_li_to_other_li(
    str_li, pattern="i", per_letter=False, sep=" ", start_row=0, end_row=None
):
    """ Convert a list of string to a list of other types
    
    pattern: a list of types for one item. 
        'i' for int, 's' for string
        'si' means: convert the 1st item to string, the rest to integer
        If separated items are more than pattern items,
        use the last one from the parttern.
    if per_letter=True, ignore sep and separate item per letter
    """
    target_str_li = str_li[start_row:end_row]
    # find max item num
    max_item_num = 1
    if per_letter:
        max_item_num = max([len(s) for s in target_str_li])
    else:
        max_item_num = max([len(s.split(sep)) for s in target_str_li])

    # extend the pattern to the max itme num
    pattern = (
        pattern + f"{pattern[-1]}" * (max_item_num - len(pattern))
        if max_item_num > len(pattern)
        else pattern
    )

    # convert
    if per_letter:
        return [
            [converts[pattern[idx]](item) for idx, item in enumerate(s)]
            for s in target_str_li
        ]
    else:
        if sep == " ":
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split())]
                for s in target_str_li
            ]
        else:
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split(sep))]
                for s in target_str_li
            ]

In [3]:
fp = "input.txt"
lines = read_file_to_str_li(fp)

print("Convert to:")

# head = convert_str_li_to_other_li(
#     lines, pattern="s", per_letter=True, sep=",", start_row=0, end_row=1
# )

# print(f"Head:\n{head}")
# print(f"First line: {head[0]}")
# print(f"Last line: {head[-1]}")

data = convert_str_li_to_other_li(
    lines, pattern="s", per_letter=True, sep=" ", start_row=None, end_row=None
)


print(f"First line: {data[0]}")
print(f"Last line: {data[-1]}")
print("-" * 6)

Read from input.txt:

First line: FqdWDFppHWhmwwzdjvjTRTznjdMv | Last line: 
gHVVnBzJcBgVhtZZtgJhpGrGNpvDpddvprNGrlTJ

------

Convert to:

First line: ['F', 'q', 'd', 'W', 'D', 'F', 'p', 'p', 'H', 'W', 'h', 'm', 'w', 'w', 'z', 'd', 
'j', 'v', 'j', 'T', 'R', 'T', 'z', 'n', 'j', 'd', 'M', 'v']

Last line: ['g', 'H', 'V', 'V', 'n', 'B', 'z', 'J', 'c', 'B', 'g', 'V', 'h', 't', 'Z', 'Z', 
't', 'g', 'J', 'h', 'p', 'G', 'r', 'G', 'N', 'p', 'v', 'D', 'p', 'd', 'd', 'v', 'p', 'r', 
'N', 'G', 'r', 'l', 'T', 'J']

------

In [4]:
ord("a"), ord("A")

(97, 65)

In [5]:
# a->1, z->26, A->27, Z->52
num_li = [[ord(l) - 96 if ord(l) >= 97 else ord(l) - 38 for l in li] for li in data]

sep_num_li = [[li[: len(li) // 2], li[len(li) // 2 :]] for li in num_li]
com_num_li = [np.intersect1d(li[0], li[1]) for li in sep_num_li]
print(f"Answer to Q1: {np.sum(com_num_li)}")

sep_num_li = [num_li[i * 3 : i * 3 + 3] for i in range(len(num_li) // 3)]
com_num_li = [reduce(np.intersect1d, (li[0], li[1], li[2])) for li in sep_num_li]
print(f"Answer to Q2: {np.sum(com_num_li)}")

Answer to Q1: 8243

Answer to Q2: 2631